## Pruning experiments

https://pytorch.org/tutorials/intermediate/pruning_tutorial.html

In [60]:
import torch
import torchvision
from torch import nn
from torchvision import models
import torch.nn.utils.prune as prune
import torch.nn.functional as F
import csv
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, ToPILImage
import numpy as np
import random

import io
import os
import pandas as pd

from torch.utils.data import Dataset

In [61]:
model_id = "uk_garden_birds_mock"
path = "20200827_uk_garden_birds_mock/"

## Load all the models

Use the model previously created (see naturewatch_pytorch_inference)

In [62]:
## unpruned model
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

with open(path+model_id+'_classes.csv', newline='') as f:
    reader = csv.reader(f)
    class_list = list(reader)[0]

# we need the correct structure to load it (I think?)
model_full = models.resnet34() #load resnet structure
num_ftrs = model_full.fc.in_features 
num_classes = len(class_list)
model_full.fc = nn.Linear(num_ftrs, num_classes) #change final layer

model_full.load_state_dict(torch.load(path+model_id+'_model.pt', map_location=map_location))
model_full.eval()
print("loaded full model")

loaded full model


In [63]:
## predictions stuff

def transform_image(image_bytes): #change to match trainging
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)


def get_predictions(image_bytes, n, model0):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model0(tensor)
    softmax = nn.Softmax(dim=1)
    probs = softmax(outputs)
    
    top_probs, top_idxs = torch.topk(probs, n)
    
    for i in range(0,n):
        print(class_list[top_idxs[0][i]], top_probs[0][i].item())
        
def time_test_prediction(path, model0):
    with open(path, 'rb') as f:
        image_bytes = f.read()
        get_predictions(image_bytes=image_bytes, n=3, model0=model0)


In [64]:
image_size = 224 # must be 224 for Resnet default
batch_size = 32
weight_decay = 0.01  # PyTorch default = 0, fastai says to try 0.1


transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # augmentation
    transforms.RandomRotation(5),
    transforms.Resize((int(image_size*1.1),int(image_size*1.1))),       # upscale if required
    transforms.CenterCrop((image_size,image_size)),   # resize
    transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

test_transform = transforms.Compose([
    transforms.Resize((image_size,image_size)),       # upscale if required
    transforms.CenterCrop((image_size,image_size)),   # resize
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

class DatasetFromCSV(Dataset):
    def __init__(self, path_csv, path_img_dir, transform=None, classes_list = None):
        df_fromCSV = pd.read_csv(path_csv, header=None)
        self.img_paths = df_fromCSV[0]
        self.img_labels = self.convert_label_strings_to_indices(df_fromCSV[1], classes_list) if classes_list is not None else df_fromCSV[1] 
        self.path_csv = path_csv
        self.path_img_dir = path_img_dir
        self.transform = transform
        self.to_tensor = ToTensor()
        self.to_pil = ToPILImage()
        self.image_size = image_size
    
    def get_image_from_folder(self, name):
        img = Image.open(os.path.join(self.path_img_dir, name))
        if img.mode == 'CMYK':
            img = img.convert('RGB')
        return img 
    
    def convert_label_strings_to_indices(self, labels_as_strings, classes):
        labels_as_indices = []
        classes = classes.tolist()
        for label in labels_as_strings:
            labels_as_indices.append(int(classes.index(label)))
        return labels_as_indices
    
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        ''' 
        returns image as normalised tensors of shape (num_channels, num_px_x, num_px_y)
        returns label as index (i.e. of type int) relative to its position in the classes array
        '''
        image = self.get_image_from_folder(self.img_paths[index])
        try:
            if self.transform is not None:
                image = self.transform(image)
        except RuntimeError as rerr:
            print (rerr, self.img_paths[index], index)
        label = self.img_labels[index] 


        return image, label
    
    def __getrandom__(self):
        '''
        like getitem but gets a _random_ image (path) and its label
        '''
        index = random.randint(0, len(self.img_paths))
        ##image = self.get_image_from_folder(self.img_paths[index])
        label = self.img_labels[index]
        
        return self.img_paths[index], label

# load subset from CSV
basePaths_csvs = "20201107_balanced_test_and_validation/lists/" # <--- adjust to your system
path_CSV_test =  basePaths_csvs  + "uk_garden_birds_balanced_test.csv"
basePath_ds = "uk_garden_birds_balanced/"
image_path_test = basePath_ds + "test"
num_workers = 0 # processing cores to use

classes = pd.read_csv(path_CSV_test, header=None)[1].unique() 

data_test = DatasetFromCSV(path_CSV_test, image_path_test, test_transform, classes)

test_loader = torch.utils.data.DataLoader(data_test, batch_size=batch_size,
     num_workers=num_workers)

In [65]:
def test_model(model, classes):
    # track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(classes.size)) # create emtpy array with slot for each class
    class_total = list(0. for i in range(classes.size))

    accuracy = []
    imagenum = []

    #track predictions and targets for evaluation
    predslist=torch.zeros(0,dtype=torch.long, device=map_location)
    imageslist=torch.zeros([0,3, image_size, image_size], device=map_location) #keep track of the order of the batches' images 
    targslist=torch.zeros(0,dtype=torch.long, device=map_location)
    losseslist=torch.zeros(0,dtype=torch.float, device=map_location)
    probslist=torch.zeros(0,dtype=torch.float, device=map_location)

    criterion = nn.CrossEntropyLoss() # outputs mean squared error for the overall batch
    criterion_perSample = nn.CrossEntropyLoss(reduction = 'none')

    model.eval()
    train_on_gpu = False

    print ("Evaluating model performance on test set ... \n")
    # iterate over test data
    for data, target in test_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target) # mse batch loss 
        losses_perSample = criterion_perSample(output, target) # individual losses per image sample
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
    
        # normalize output to probabalites adding to 1 with softmax 
        softmax = nn.Softmax(dim=1)
        probs = softmax(output)
    
        # add predictions and targets to list
        predslist=torch.cat([predslist,pred.view(-1).cpu()])
        imageslist= torch.cat([imageslist,data.data.cpu()])
        targslist=torch.cat([targslist,target.view(-1).cpu()])
        losseslist=torch.cat([losseslist, losses_perSample.data.cpu()])
        probslist=torch.cat([probslist, probs.data.cpu()])
    
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
    
        for i in range(target.data.size()[0]): # for every sample in current batch (last batch might not be full batchsize if n_X % batch_size != 0)
            label = target.data[i]
            try:
                class_correct[label] += correct[i].item()
            except:
                class_correct[label] += correct
            class_total[label] += 1

    # average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('Test Accuracy (Overall): %2d%% (%2d/%2d)' % (
         100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

## No pruning

In [66]:
test_model(model_full, classes)

Evaluating model performance on test set ... 

Test Loss: 1.739588

Test Accuracy (Overall): 54% (757/1400)


In [80]:
%%time
p = 'test/Eurasian Sparrowhawk/15bccfd131deb7f7334771e1d3771944.jpg'
time_test_prediction(p, model_full)

House Sparrow 0.13860872387886047
Eurasian Sparrowhawk 0.12889806926250458
Long-tailed Tit 0.12249056994915009
CPU times: user 136 ms, sys: 18.3 ms, total: 155 ms
Wall time: 155 ms


## Prune 1

pruning_method=prune.L1Unstructured

In [68]:
## prune 1 - 
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

with open(path+model_id+'_classes.csv', newline='') as f:
    reader = csv.reader(f)
    class_list = list(reader)[0]

# we need the correct structure to load it (I think?)
model_prune1 = models.resnet34() #load resnet structure
num_ftrs = model_prune1.fc.in_features 
num_classes = len(class_list)
model_prune1.fc = nn.Linear(num_ftrs, num_classes) #change final layer

model_prune1.load_state_dict(torch.load(path+model_id+'_model.pt', map_location=map_location))
model_prune1.eval()
print("loaded prune 1 model")

#https://discuss.pytorch.org/t/module-children-vs-module-modules/4551
#parameters_to_prune = [(child, "weight") for child in model.modules()]
parameters_to_prune = [
    (child, "weight")
    for child in model_prune1.modules()
    if (isinstance(child, torch.nn.Conv2d) or isinstance(child, torch.nn.BatchNorm2d))
]

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.5,
)
test_model(model_prune1, classes)

loaded prune 1 model
Evaluating model performance on test set ... 

Test Loss: 2.079975

Test Accuracy (Overall): 46% (656/1400)


In [82]:
%%time
p = 'test/Eurasian Sparrowhawk/15bccfd131deb7f7334771e1d3771944.jpg'
time_test_prediction(p, model_prune1)

House Sparrow 0.02619250863790512
European Starling 0.021677931770682335
Mallard 0.020930804312229156
CPU times: user 162 ms, sys: 25.9 ms, total: 187 ms
Wall time: 177 ms


## Prune 2

test pruning by weights theshold
https://stackoverflow.com/questions/61629395/how-to-prune-weights-less-than-a-threshold-in-pytorch

see some more stuff here https://stackoverflow.com/questions/62326683/prunning-model-doesnt-improve-inference-speed-or-reduce-model-size

it may not improve inference speed

In [86]:
#reset
# reset dosn't actually reset. need to load it all again
if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

with open(path+model_id+'_classes.csv', newline='') as f:
    reader = csv.reader(f)
    class_list = list(reader)[0]

# we need the correct structure to load it (I think?)
model_prune2 = models.resnet34() #load resnet structure
num_ftrs = model_prune2.fc.in_features 
num_classes = len(class_list)
model_prune2.fc = nn.Linear(num_ftrs, num_classes) #change final layer

model_prune2.load_state_dict(torch.load(path+model_id+'_model.pt', map_location=map_location))
model_prune2.eval()

print("loaded prune 2 model")
from torch.nn.utils import prune

class ThresholdPruning(prune.BasePruningMethod):
    PRUNING_TYPE = "unstructured"

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
        #print("self.threshold...",self.threshold)
        #print("tttt",torch.abs(tensor) > self.threshold)
        return torch.abs(tensor) > self.threshold
    
# weights seem to be be between 1 and 2
prune.global_unstructured(
    parameters_to_prune, pruning_method=ThresholdPruning, threshold=1.1
)
test_model(model_prune2, classes)

loaded prune 2 model
Evaluating model performance on test set ... 

Test Loss: 1.739588

Test Accuracy (Overall): 54% (757/1400)


In [87]:
%%time
p = 'test/Eurasian Sparrowhawk/15bccfd131deb7f7334771e1d3771944.jpg'
time_test_prediction(p, model)

Mallard 0.06670695543289185
Common Wood Pigeon 0.04748813435435295
Coal Tit 0.032256465405225754
CPU times: user 197 ms, sys: 55.7 ms, total: 253 ms
Wall time: 246 ms
